In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib inline

#for working on time and dates
import datetime as dt
import time

#yahoo finance lib
import yfinance as yf

#for handling files
import os 
from os import listdir
from os.path import isfile, join

#import plotly lib
import cufflinks as cf
import plotly as ply
import plotly.express as px
import plotly.graph_objects as go

#Make Plotly work in jupyter notebook
ply.offline.init_notebook_mode(connected=True)
    #ply.offline(parameter) connected: Literal[True] ook_mode, plot, iplot
    #init_notebook_mode(connected=True)

#use plotly locally
cf.go_offline()

from plotly.subplots import make_subplots

import warnings
warnings.simplefilter('ignore')

## Constants

In [2]:
Path = "C:/Users/davin/Documents/Data Science/My Projects/Untitled Folder/My Portfolio/"

#start end date defaults
s_date = '2017-02-01'
e_date = '2022-12-06'
s_date_dt = pd.to_datetime(s_date)
e_date_dt = pd.to_datetime(e_date)


## Column Data

In [3]:
def get_column_from_csv(file, col_name):
    
    try:
        df = pd.read_csv(file,delimiter=';')
    except FileNotFoundError:
        print("File Doesn't Exist")
    else:
        return df[col_name]

In [4]:
tickers = get_column_from_csv('tickers.csv','Ticker')
tickers

0     PTBA.JK
1     UNVR.JK
2     BBCA.JK
3     BMRI.JK
4     BBRI.JK
5     GGRM.JK
6     ITMG.JK
7     UNTR.JK
8     ADMF.JK
9     SMGR.JK
10    INTP.JK
Name: Ticker, dtype: object

## Download Stocks Data

In [1]:
stocks_not_downloaded =[]
missing_stocks = []

In [5]:
def save_stock_to_csv(folder,tickers):
    stock = yf.Ticker(tickers)
    try:
        print('Get Data for :', tickers)
        df = stock.history(period = '5y')
        time.sleep(2)
        if df.empty:
            stocks_not_downloaded.append(tickers)
        the_file = folder + tickers.replace('.','_')+'.csv'
        print(the_file,'Saved')
        df.to_csv(the_file)
    except Exception as ex:
        stocks_not_downloaded.append(tickers)
        print('Couldnt get data for:',tickers)

In [6]:
folder = "C:/Users/davin/Documents/Data Science/My Projects/Untitled Folder/My Portfolio/"
for x in range(10):
    save_stock_to_csv(folder,tickers[x])
print('Finished')

Get Data for : PTBA.JK
C:/Users/davin/Documents/Data Science/My Projects/Untitled Folder/My Portfolio/PTBA_JK.csv Saved
Get Data for : UNVR.JK
C:/Users/davin/Documents/Data Science/My Projects/Untitled Folder/My Portfolio/UNVR_JK.csv Saved
Get Data for : BBCA.JK
C:/Users/davin/Documents/Data Science/My Projects/Untitled Folder/My Portfolio/BBCA_JK.csv Saved
Get Data for : BMRI.JK
C:/Users/davin/Documents/Data Science/My Projects/Untitled Folder/My Portfolio/BMRI_JK.csv Saved
Get Data for : BBRI.JK
C:/Users/davin/Documents/Data Science/My Projects/Untitled Folder/My Portfolio/BBRI_JK.csv Saved
Get Data for : GGRM.JK
C:/Users/davin/Documents/Data Science/My Projects/Untitled Folder/My Portfolio/GGRM_JK.csv Saved
Get Data for : ITMG.JK
C:/Users/davin/Documents/Data Science/My Projects/Untitled Folder/My Portfolio/ITMG_JK.csv Saved
Get Data for : UNTR.JK
C:/Users/davin/Documents/Data Science/My Projects/Untitled Folder/My Portfolio/UNTR_JK.csv Saved
Get Data for : ADMF.JK
C:/Users/davin/Do

## Get all stocks in a list

In [8]:
files = [x for x in listdir(Path) if isfile(join(Path,x))]
tickers = [os.path.splitext(x)[0] for x in files]
tickers.sort()
print(tickers)
print(len(tickers))

['ADMF_JK', 'BBCA_JK', 'BBRI_JK', 'BMRI_JK', 'GGRM_JK', 'ITMG_JK', 'PTBA_JK', 'SMGR_JK', 'UNTR_JK', 'UNVR_JK']
10


## daily return

In [9]:
def daily_return_to_df(df):
    df['daily_return'] = (df['Close']/df['Close'].shift(1))-1
    #df.to_csv(Path + ticker + '.csv')
    return df

## cumulative return

In [10]:
def cum_return_to_df(df):
    df['cum_return'] = (1+df['daily_return']).cumprod()
    #df.to_csv(Path + ticker + '.csv')
    return df

## Bolinger bands

Bollinger Bands plot 2 lines using a moving average and the standard deviation defines how far apart the lines are. They also are used to define if prices are to high or low. When bands tighten it is believed a sharp price move in some direction. Prices tend to bounce off of the bands which provides potential market actions.

A strong trend should be noted if the price moves outside the band. If prices go over the resistance line it is in overbought territory and if it breaks through support it is a sign of an oversold position.

In [11]:
def bollinger_bands(df):
    df['middle_band'] = df['Close'].rolling(window = 20).mean()
    df['upper_band'] = df['middle_band']+ 1.96 * df['Close'].rolling(window = 20).std()
    df['lower_band'] = df['middle_band']- 1.96 * df['Close'].rolling(window = 20).std()
    #df.to_csv(Path + ticker + '.csv')
    return df

## Ichimoku

The Ichimoku (One Look) is considered an all in one indicator. It provides information on momentum, support and resistance. It is made up of 5 lines. If you are a short term trader you create 1 minute or 6 hour. Long term traders focus on day or weekly data.

- Conversion Line (Tenkan-sen) : Represents support, resistance and reversals. Used to measure short term trends.
- Baseline (Kijun-sen) : Represents support, resistance and confirms trend changes. Allows you to - -evaluate the strength of medium term trends. Called the baseline because it lags the price.
- Leading Span A (Senkou A) : Used to identify future areas of support and resistance
- Leading Span B (Senkou B) : Other line used to identify suture support and resistance
- Lagging Span (Chikou) : Shows possible support and resistance. It is used to confirm signals obtained from other lines.
- Cloud (Kumo) : Space between Span A and B. Represents the divergence in price evolution.


Formulas

- Lagging Span = Price shifted back 26 periods
- Base Line = (Highest Value in period + Lowest value in period)/2 (26 Sessions)
- Conversion Line = (Highest Value in period + Lowest value in period)/2 (9 Sessions)
- Leading Span A = (Conversion Value + Base Value)/2 (26 Sessions)
- Leading Span B = (Conversion Value + Base Value)/2 (52 Sessions)

In [12]:
def Ichimoku(df):
    # Conversion = (highest value in period + lowest value in period)/2 (9 sessions)
    hi_val = df['High'].rolling(window=9).max()
    low_val = df['Low'].rolling(window=9).min()
    df['Conversion'] = (hi_val + low_val) / 2

    # Baseline = (highest value in period + lowest value in period)/2 (26 sessions)
    hi_val2 = df['High'].rolling(window=26).max()
    low_val2 = df['Low'].rolling(window=26).min()
    df['Baseline'] = (hi_val2 + low_val2) / 2

    # Spans = ()
    df['SpanA'] = ((df['Conversion'] + df['Baseline']) / 2).shift(26)
    hi_val3 = df['High'].rolling(window=52).max()
    low_val3 = df['Low'].rolling(window=52).min()
    df['SpanB'] = ((hi_val3 + low_val3) / 2).shift(26)
    df['Lagging'] = df['Close'].shift(-26)

    return df

## Get Dataframe from CSV

In [7]:
def get_stock_df_from_csv(ticker):
    try:
        df = pd.read_csv(Path + ticker + '.csv', index_col=0)
    except FileNotFoundError as ex:
        Print('File Not Found')
    else:
        return df

## Add Daily, Cumulative Bollinger Bands & Ichimoku to Dataframes

In [15]:
for x in tickers:
    try:
        print("Working on :", x)
        new_df = get_stock_df_from_csv(x)
        new_df = daily_return_to_df(new_df)
        new_df = cum_return_to_df(new_df)
        new_df = bollinger_bands(new_df)
        new_df = Ichimoku(new_df)
        new_df.to_csv(Path + x + '.csv')
    except Exception as ex:
        print(ex)

Working on : ADMF_JK
Working on : BBCA_JK
Working on : BBRI_JK
Working on : BMRI_JK
Working on : GGRM_JK
Working on : ITMG_JK
Working on : PTBA_JK
Working on : SMGR_JK
Working on : UNTR_JK
Working on : UNVR_JK


In [14]:
# It is always better to test your results using one stock
# try:
#     print("Working on :", "A")
#     new_df = get_stock_df_from_csv("A")
#     new_df = add_daily_return_to_df(new_df)
#     new_df = add_cum_return_to_df(new_df)
#     new_df = add_bollinger_bands(new_df)
#     new_df = add_Ichimoku(new_df)
#     new_df.to_csv(PATH + 'A' + '.csv')
# except Exception as ex:
#     print(ex)